<a href="https://colab.research.google.com/github/Interpause/pseudo-text/blob/master/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports & setup

In [0]:
import numpy as np
import pandas as pd

import nltk
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.svm import SVC
from sklearn import metrics

from collections import OrderedDict
import string, json

In [2]:
from google.colab import drive #drive.flush_and_unmount() #drive.flush()
import os
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/CSIT Internship/code')
nltk.download('all',quiet=True)
!nvidia-smi

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Wed Nov 27 02:42:08 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.33.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P8    29W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+--------------------------------------

# Cleaning Functions

In [0]:
# List of all words to remove from statements
removables = []
removables += nltk.corpus.stopwords.words('english')
removables += [w.lower() for w in nltk.corpus.names.words()]
removables += ['hi','trump']

lemmatizer = nltk.stem.WordNetLemmatizer()
#data fed to vectorizer is already cleaned but maybe TODO use their params instead
vectorizer = TfidfVectorizer(max_features=400) #gets kwords on principle of very used in doc, but rare across all docs
#vectorizer = CountVectorizer(max_features=400) #gets kwords by frequency distribution (is not smart)

In [0]:
#much info was lost
#some of these could be specified as options to the Tfidf vectorizer instead
def filterSent(s):
    s = s.lower()
    s = nltk.word_tokenize(s) #split into list of words intelligently
    s = tuple(lemmatizer.lemmatize(w) for w in s if w.isalpha()) #supposed to lemmatize words (aka get basic form)
    s = tuple(w for w in s if w not in removables) #remove removables
    return ' '.join(s)
filterSent("hi there my name is jOhn henry battery car dead better programming")

#applied to dataset to reformat data
formatter = lambda x:(filterSent(x.statement),liar_score[x.label])

def printScore(y_pred,y_test):
    #assuming balanced, otherwise using balanced_accuracy_score()
    print("Accuracy: {0:.1%}".format(metrics.accuracy_score(y_test, y_pred)))
    #TODO add more

# LIAR dataset

In [0]:
path_to_liar = './raw/liar_dataset/'
#counts refer to the speaker's reliability as a whole, interestingly excluding true counts. 
liar_headers = ('id','label','statement',
                'subjects','speaker','job_title',
                'state','party','barely_true_counts',
                'false_counts','half_true_counts','mostly_true_counts',
                'pants_on_fire_counts','context')

liar_score = {'true':1,
              'mostly-true':1,
              'half-true':1,
              'barely-true':0,
              'false':0,
              'pants-fire':0
             }

In [0]:
#Opens dataset, extracts only statement & label and reformats them using formatter()
liar_train = pd.read_csv(path_to_liar+'train.tsv',delimiter='\t',header=None,names=liar_headers)
traindf = liar_train.loc[:,('statement','label')].apply(formatter,axis=1,result_type='broadcast')
liar_test = pd.read_csv(path_to_liar+'test.tsv',delimiter='\t',header=None,names=liar_headers)
testdf = liar_test.loc[:,('statement','label')].apply(formatter,axis=1,result_type='broadcast')

# Bag of Words + SVM approach

In [7]:
#trains vectorizer on training dataset, followed by transforming training dataset into vectors
train_data = vectorizer.fit_transform(traindf.statement)
x_train = train_data.toarray()
y_train = np.array(traindf.label,dtype=int)
#transforms test dataset into vectors without training
test_data = vectorizer.transform(testdf.statement)
x_test = test_data.toarray()
y_test = np.array(testdf.label, dtype=int)
#see which keywords were extracted as features
print(vectorizer.get_feature_names())

['abortion', 'according', 'act', 'actually', 'administration', 'affordable', 'agency', 'ago', 'allow', 'allowed', 'almost', 'already', 'also', 'amendment', 'america', 'american', 'among', 'amount', 'attack', 'average', 'away', 'back', 'background', 'ban', 'bank', 'barack', 'benefit', 'big', 'billion', 'black', 'board', 'border', 'break', 'brown', 'budget', 'bush', 'business', 'buy', 'california', 'called', 'came', 'campaign', 'candidate', 'cant', 'car', 'care', 'cent', 'change', 'check', 'child', 'china', 'citizen', 'city', 'class', 'college', 'combined', 'come', 'community', 'company', 'congress', 'congressional', 'congressman', 'control', 'cost', 'could', 'country', 'county', 'court', 'coverage', 'create', 'created', 'credit', 'crime', 'criminal', 'current', 'cut', 'day', 'deal', 'death', 'debate', 'debt', 'decade', 'defense', 'deficit', 'democrat', 'democratic', 'department', 'didnt', 'district', 'doesnt', 'dollar', 'done', 'dont', 'doubled', 'drug', 'due', 'economic', 'economy', 'e

In [8]:
#fits SVM on vectorized training dataset
clf = SVC(kernel='linear')
clf.fit(x_train,y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='linear', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [9]:
y_pred = clf.predict(x_test)
printScore(y_pred,y_test)

Accuracy: 60.8%


# ELMo Sum approach

In [0]:
sentences[20 < sentences.map(len) < 50]

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [0]:
'''
from allennlp.modules.elmo import Elmo, batch_to_ids
options_file = "./elmo_2x4096_512_2048cnn_2xhighway_options.json"
weight_file = "./elmo_2x4096_512_2048cnn_2xhighway_weights.hdf5"
elmo = Elmo(options_file, weight_file, 1, dropout=0) #1 representation only
'''

In [0]:
'''
#I would try this if i hadnt almost immediately crashed from lag. maybe its cause its padded. ill take data points within a length range then
sentences = traindf.statement.str.split()
character_ids = batch_to_ids(sentences)
embeddings = elmo(character_ids) #[representation][sent_no][word_no (padded)]
'''

RuntimeError: $ Torch: not enough memory: you tried to allocate 7GB. Buy new RAM! at ..\aten\src\TH\THGeneral.cpp:201

['abortion',
 'according',
 'act',
 'actually',
 'administration',
 'affordable',
 'agency',
 'ago',
 'allow',
 'allowed',
 'almost',
 'already',
 'also',
 'amendment',
 'america',
 'american',
 'among',
 'amount',
 'attack',
 'average',
 'away',
 'back',
 'background',
 'ban',
 'bank',
 'barack',
 'benefit',
 'big',
 'billion',
 'black',
 'board',
 'border',
 'break',
 'brown',
 'budget',
 'bush',
 'business',
 'buy',
 'california',
 'called',
 'came',
 'campaign',
 'candidate',
 'cant',
 'car',
 'care',
 'cent',
 'change',
 'check',
 'child',
 'china',
 'citizen',
 'city',
 'class',
 'college',
 'combined',
 'come',
 'community',
 'company',
 'congress',
 'congressional',
 'congressman',
 'control',
 'cost',
 'could',
 'country',
 'county',
 'court',
 'coverage',
 'create',
 'created',
 'credit',
 'crime',
 'criminal',
 'current',
 'cut',
 'day',
 'deal',
 'death',
 'debate',
 'debt',
 'decade',
 'defense',
 'deficit',
 'democrat',
 'democratic',
 'department',
 'didnt',
 'district',

In [0]:
fmt_string = '{:f}={}'
ndata = []
with open('test.txt','w') as f:
    for _,row in df.iterrows():
        tup = tuple(w for w in row.statement if w in kept)
        if len(tup) < 10: continue
        print(fmt_string.format(row.label,json.dumps(tup)),file=f)
        ndata.append((tup,row.label))

In [0]:
fdata = {
    'w2idx':dict((k,v) for v,k in enumerate(kept)),
    'data':ndata
}

In [0]:
with open('test.json','w') as f:
    json.dump(fdata,f)

In [0]:
maxlen = 0
largest = None
for tup,t in fdata['data']:
    if len(tup) > maxlen:
        maxlen = len(tup)
        largest = tup

In [0]:
largest

('hospital',
 'doctor',
 'used',
 'far',
 'country',
 'many',
 'country',
 'former',
 'governor',
 'massachusetts',
 'republican',
 'news',
 'obamacare',
 'cut',
 'senior',
 'medicare',
 'medicare',
 'republican',
 'republican',
 'campaign',
 'email',
 'many',
 'federal',
 'employee',
 'cost',
 'taxpayer',
 'million',
 'spends',
 'million',
 'year',
 'child',
 'education',
 'democrat',
 'campaign',
 'milwaukee',
 'county',
 'citizen',
 'point',
 'center',
 'mass',
 'best',
 'option',
 'crime',
 'gun',
 'wisconsin',
 'campaign',
 'ad',
 'almost',
 'percent',
 'total',
 'income',
 'planned',
 'parenthood',
 'abortion',
 'abortion',
 'state',
 'representative',
 'republican',
 'committee',
 'united',
 'state',
 'ha',
 'highest',
 'rate',
 'poverty',
 'almost',
 'country',
 'earth',
 'child',
 'poverty',
 'senator',
 'independent',
 'democratic',
 'debate',
 'governor',
 'program',
 'allowed',
 'undocumented',
 'people',
 'use',
 'personal',
 'tax',
 'number',
 'receive',
 'driver',
 'lice